In [2]:
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
from PIL import Image
import numpy as np
import os
from pathlib import Path
import pdb
import json

In [34]:
def ground_truth_np(file_name):
    js  = '/home/huyentn2/project/nano_count/segmentation_unet/inference/gt_all_count.json'

    with open(js, 'r') as fp:
        gt_count = json.load(fp)

    if file_name in list(gt_count.keys()):
        return gt_count[file_name]
    return None

In [35]:
# ground_truth_np('Tianle60min-1fm375-aqu-DSC_2.png')

186

In [8]:
im = np.array(Image.open(input_dir))
im.max()

208

In [31]:
def get_peaks(input, min_dist, thr):
    im = np.array(Image.open(input))

    # image_max is the dilation of im with a 20*20 structuring element
    # It is used within peak_local_max function
    image_max = ndi.maximum_filter(im, size=5, mode='constant')

    # Comparison between image_max and im to find the coordinates of local maxima
    coordinates = peak_local_max(im, min_distance=min_dist, threshold_abs=thr)

    print(input)
    print("Num of peaks", len(coordinates))

    # # display results
    fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
    ax = axes.ravel()
    ax[0].imshow(im, cmap=plt.cm.gray)
    ax[0].axis('off')
    ax[0].set_title('Original')

    ax[1].imshow(image_max, cmap=plt.cm.gray)
    ax[1].axis('off')
    ax[1].set_title('Maximum filter')

    ax[2].imshow(im, cmap=plt.cm.gray)
    ax[2].autoscale(False)
    # ax[2].plot(coordinates[:, 1], coordinates[:, 0], 'r.')
    ax[2].scatter(coordinates[:, 1], coordinates[:, 0], s = 0.5, c = 'r')
    ax[2].axis('off')
    ax[2].set_title('Peak local max')

    fig.tight_layout()

    plt.show()

    return len(coordinates)

In [ ]:
input_dir = "/home/huyentn2/project/nano_count/segmentation_unet/checkpoints_7_4/save_result/gray_pred_inv/fold0/Yang40min-1fm375-aqDMSO-DSC_3.png"
print("GT", ground_truth_np('Yang40min-1fm375-aqDMSO-DSC_3.png'))
min_dist = 10
thr = 150

get_peaks(input_dir, min_dist, thr)

In [1]:
np.random.seed(100)

input_dir = "/home/huyentn2/project/nano_count/segmentation_unet/save_path/save_result/gray_pred_inv/fold0/"

def local_max_peaks(input, min_dist, thr):
    im = np.array(Image.open(input))

    # image_max is the dilation of im with a 20*20 structuring element
    # It is used within peak_local_max function
    image_max = ndi.maximum_filter(im, size=20, mode='constant')

    # Comparison between image_max and im to find the coordinates of local maxima
    coordinates = peak_local_max(im, min_distance=min_dist, threshold_abs=thr)

    print(input)
    print("Num of peaks", len(coordinates))

    # # display results
    # fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
    # ax = axes.ravel()
    # ax[0].imshow(im, cmap=plt.cm.gray)
    # ax[0].axis('off')
    # ax[0].set_title('Original')

    # ax[1].imshow(image_max, cmap=plt.cm.gray)
    # ax[1].axis('off')
    # ax[1].set_title('Maximum filter')

    # ax[2].imshow(im, cmap=plt.cm.gray)
    # ax[2].autoscale(False)
    # ax[2].plot(coordinates[:, 1], coordinates[:, 0], 'r.')
    # ax[2].axis('off')
    # ax[2].set_title('Peak local max')

    # fig.tight_layout()

    # plt.show()

    return len(coordinates)

In [2]:
def get_count(input_dir, min_dist, thr):
    dir = input_dir
    file_list = os.listdir(dir)

    count_pred = []
    dict_pred = {}

    for file in file_list:
        input = dir + file
        num_peaks = local_max_peaks(input, min_dist, thr)
        count_pred.append(num_peaks)
        dict_pred[file] = num_peaks

    return count_pred, dict_pred


In [3]:
def plot(input_dir, count_pred, count_gt, min_dist, thr, mae):
    fig1 = plt.figure()
    ax = plt.axes()
    x = np.linspace(0, len(count_gt)-1, len(count_gt))
    ax.plot(x, count_gt, 'b', label= 'gt')
    ax.plot(x, count_pred, 'r', label= 'pred')

    plt.xlabel('Image',fontsize=20,fontname='serif')
    plt.ylabel('Num of particles',fontsize=20,fontname='serif')

    plt.title("Threshold {} and Min distance {} : MAE {:.2f}".format(thr, min_dist, mae))

    font1 = {'family': 'serif',
    'weight': 'normal',
    'size': 7,
    }

    handles, labels = plt.gca().get_legend_handles_labels()
    order = [len(handles)-x for x in range(1,len(handles)+1)]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order],loc='lower left', prop=font1)
    plt.grid(linestyle='--')
    fig1.savefig(input_dir + 'Counting_performace_th{}_d{}'.format(thr, min_dist),bbox_inches='tight',dpi=300)
    plt.clf()


In [8]:
def single_img_count():
    import json
    js  = '/home/huyentn2/project/nano_count/segmentation_unet/inference/gt_all_count.json'

    with open(js, 'r') as fp:
        gt_count = json.load(fp)

    count_gt = []
    dict_gt = {}
    for file in os.listdir(input_dir):
        count = gt_count[file]
        count_gt.append(count)
        dict_gt[file] = count


    # min_dist = 20
    # thr = 125

    min_dist_list = [5, 10, 15, 20, 25, 30]
    thr_list = [75, 90, 100, 110, 120, 125, 130, 145, 150]

    mae_dict = {}

    for min_dist in min_dist_list:
        for thr in thr_list:

            count_pred, _  = get_count(input_dir, min_dist, thr)

            a = np.array(count_gt)
            b = np.array(count_pred)
            mae = np.sum(np.abs(a - b))/ len(a)

            mae_dict['thr_{}_dist_{}'.format(thr, min_dist)] = mae

            # plot(input_dir, count_pred, count_gt, min_dist, thr, mae)

    # pdb.set_trace()
    min = mae_dict[list(mae_dict.keys())[0]]
    key = None
    for k in mae_dict:
        if mae_dict[k] < min:
            min = mae_dict[k]
            key = k

    print(key, min)

In [9]:
if __name__ == "__main__":
    # import json
    # js  = '/home/huyentn2/huyen/project/data_nano/gt_all_count.json'

    # with open(js, 'r') as fp:
    #     gt_count = json.load(fp)

    # dir = "/home/huyentn2/huyen/project/project_nano_count/segmentation_unet_/save_results/save_prediction_salmon/"
    # folds = ['fold0/', 'fold1/', 'fold2/', 'fold3/', 'fold4/', 'fold5/', 'fold6/', 'fold7/', 'fold8/', 'fold9/']

    # input_dir_list = [dir + choice for choice in folds]

    # for input_d in input_dir_list:
    #     count_gt = []
    #     dict_gt = {}
    #     for file in os.listdir(input_d + 'rs/'):
    #         count = gt_count[file]
    #         count_gt.append(count)
    #         dict_gt[file] = count


    #     # min_dist = 20
    #     # thr = 125

    #     min_dist_list = [5, 10, 15, 20, 25, 30]
    #     thr_list = [75, 90, 100, 110, 120, 125, 130, 145, 150]

    #     mae_dict = {}

    #     for min_dist in min_dist_list:
    #         for thr in thr_list:

    #             count_pred, _  = get_count(input_d, min_dist, thr)

    #             a = np.array(count_gt)
    #             b = np.array(count_pred)
    #             mae = np.sum(np.abs(a - b))/ len(a)

    #             mae_dict['thr_{}_dist_{}'.format(thr, min_dist)] = mae

    #             # plot(input_d, count_pred, count_gt, min_dist, thr, mae)
        
    #     min = mae_dict[list(mae_dict.keys())[0]]
    #     key = None
    #     for k in mae_dict:
    #         if mae_dict[k] < min:
    #             min = mae_dict[k]
    #             key = k

    #     # pdb.set_trace()

    #     m = int(key[4:7])
    #     th = int(key[13:])
    #     plot(input_d, count_pred, count_gt, m, th, min)

    #     print("counting of fold:" , input_d)
    #     print(key, min)
    




    single_img_count()


FileNotFoundError: [Errno 2] No such file or directory: '/home/huyentn2/project/nano_count/segmentation_unet/save_path/save_result/gray_pred_inv/fold0/rs/'